In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import Formatter
import datetime as dt
import pandas as pd

from stable_baselines.common.policies import MlpLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ppo2 import PPO2
from stable_baselines import bench, logger
from stable_baselines.results_plotter import load_results, ts2xy

from importlib import reload
from time import time
import os.path

/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
# Define your parameters here
episode_len = 120
# file_dir = "D:/School Related/Columbia/Internship/XMonetae/rl_project"

# Training

In [3]:
# train
env = gym.make('gym_trading_4995:Equity-train-v0')
env = bench.Monitor(env, logger.get_dir(), allow_early_resets=True)
env = DummyVecEnv([lambda: env])
model = PPO2(MlpLstmPolicy, env, n_steps=240, learning_rate=0.000001, verbose=0, nminibatches=1, 
             policy_kwargs={"n_lstm":36, "layers":[36,36]}, tensorboard_log="./equity_train_tensorboard/")





Instructions for updating:
Use keras.layers.flatten instead.



/home/jupyter/.local/lib/python3.5/site-packages/stable_baselines/common/policies.py:415: UserWarning: The layers parameter is deprecated. Use the net_arch parameter instead.
  warnings.warn("The layers parameter is deprecated. Use the net_arch parameter instead.")



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [4]:
env = gym.make('gym_trading_4995:Equity-train-v0')
env = DummyVecEnv([lambda: env])
model_train_continue = PPO2.load("2bps_trans_cost_36dim_state_100000", env)

ValueError: Error: the file 2bps_trans_cost_36dim_state_100000 could not be found

In [ ]:
start = time()
model_train_continue.learn(total_timesteps=episode_len*150000, tb_log_name="2bps_trans_cost_36dim_state_250000")
end = time()
t = end - start
print(f"Takes {t}s to run")

In [ ]:
model_train_continue.save("2bps_trans_cost_36dim_state_250000")

In [ ]:
class MyFormatter(Formatter):
    def __init__(self, dates, fmt='%Y-%m-%d'):
        self.dates = dates
        self.fmt = fmt

    def __call__(self, x, pos=0):
        'Return the label for time x at position pos'
        ind = int(x)
        if ind >= len(self.dates) or ind < 0:
            return ''
        else:
            return self.dates[ind].strftime(self.fmt)

def evaluate(model, environment, num_steps=40000):
    pnl = []
    dates = []
    trans_cost = []
    action_ls = []
    obs = environment.reset()
    for i in range(num_steps):
        # _states are only useful when using LSTM policies
        action, _states = model.predict(obs, deterministic=True)
        # here, action, rewards and dones are arrays, because we are using vectorized env
        obs, rewards, dones, info = environment.step(action)
        date = info[0]["date"].to_pydatetime()
        cost = info[0]["transaction_cost"]
        # Stats
        pnl.append(rewards[0])
        dates.append(date)
        trans_cost.append(cost)
        action_ls.append(action)
        if dones[0]:
            break
    
    pnl = np.array(pnl)
    dates = np.array(dates)
    actions = np.array(action_ls)
  
    return pnl, dates, trans_cost, actions

def annual_sharpe(pnl):
    mean = pnl.mean()
    var = pnl.std()
    day_sharpe = (mean / var) * np.sqrt(390)
    year_sharpe = day_sharpe * np.sqrt(252)
    return year_sharpe

def annual_return(pnl, principal=1000000):    
    ret = pnl / principal
    return np.mean(ret) * 390 * 252

def annual_volatility(pnl, principal=1000000):
    log_ret = np.log(1 + pnl / principal)
    return log_ret.std() * np.sqrt(252)

def maximum_drawdown(pnl):
    cum_pnl = np.cumsum(pnl)
    ind = np.argmax(np.maximum.accumulate(cum_pnl) - cum_pnl)
    return (np.maximum.accumulate(cum_pnl)[ind] - cum_pnl[ind]) / np.maximum.accumulate(cum_pnl)[ind]

def annual_turnover(weights):
    turnover = np.sum(np.abs(weights[1:] - weights[:-1])) / weights.shape[0]
    return turnover * 390 * 252

# 2. In-Sample test: no transaction cost

In [ ]:
env_in_0 = gym.make('Equity-insample-0bps-v0')
env_in_0 = DummyVecEnv([lambda: env_in_0])
model_train_0 = PPO2.load("no_trans_cost_36dim_state_100000", env_in_0)

In [ ]:
# Train reward before training
pnl, dates, costs, actions = evaluate(model_train_0, env_in_0, num_steps=40000)

In [ ]:
plt.style.use("ggplot")
formatter = MyFormatter(dates)
fig, ax = plt.subplots(figsize=(11, 7))
ax.xaxis.set_major_formatter(formatter)
ax.plot(np.arange(pnl.shape[0]), np.cumsum(pnl))
fig.autofmt_xdate()
plt.show()

### return statistics

In [ ]:
annual_sharpe(pnl)

In [ ]:
annual_return(pnl)

In [ ]:
annual_volatility(pnl)

In [ ]:
maximum_drawdown(pnl)

In [ ]:
annual_turnover(actions)

# 3. In Sample Test: 2bps transaction cost

In [ ]:
env_in_2 = gym.make('Equity-insample-2bps-v0')
env_in_2 = DummyVecEnv([lambda: env_in_2])
model_in_2_bad = PPO2.load("no_trans_cost_36dim_state_100000", env_in_2)

In [ ]:
pnl1, dates1, costs1, actions1 = evaluate(model_in_2_bad, env_in_2, num_steps=40000)

In [ ]:


plt.style.use("ggplot")
formatter = MyFormatter(dates1)
fig, ax = plt.subplots(figsize=(11, 7))
ax.xaxis.set_major_formatter(formatter)
ax.plot(np.arange(pnl1.shape[0]), np.cumsum(pnl1))
fig.autofmt_xdate()
plt.show()

### return statistics

In [ ]:
annual_sharpe(pnl1)

In [ ]:
annual_return(pnl1)

In [ ]:
annual_volatility(pnl1)

In [ ]:
maximum_drawdown(pnl1)

In [ ]:
annual_turnover(actions)

# 4. In sample test, 2bps cost, trained with trx cost

In [ ]:
env_in_2 = gym.make('Equity-insample-2bps-v0')
env_in_2 = DummyVecEnv([lambda: env_in_2])
model_in_2_good = PPO2.load("2bps_trans_cost_36dim_state_100000", env_in_2)

In [ ]:
pnl2, dates2, costs2, actions2 = evaluate(model_in_2_good, env_in_2, num_steps=40000)

In [ ]:
plt.style.use("ggplot")
formatter = MyFormatter(dates2)
fig, ax = plt.subplots(figsize=(11, 7))
ax.xaxis.set_major_formatter(formatter)
ax.plot(np.arange(pnl2.shape[0]), np.cumsum(pnl2))
fig.autofmt_xdate()
plt.show()

#### return statistics

In [ ]:
annual_return(pnl2)

In [ ]:
annual_volatility(pnl2)

In [ ]:
annual_sharpe(pnl2)

In [ ]:
maximum_drawdown(pnl2)

In [ ]:
annual_turnover(actions2)

## Strip transaction cost and see how it goes

In [ ]:
env_in_0 = gym.make('Equity-insample-0bps-v0')
env_in_0 = DummyVecEnv([lambda: env_in_0])
model_in_0_good = PPO2.load("2bps_trans_cost_36dim_state_100000", env_in_0)

In [ ]:
pnl3, dates3, costs3, actions3 = evaluate(model_in_0_good, env_in_0, num_steps=40000)

In [ ]:
plt.style.use("ggplot")
formatter = MyFormatter(dates3)
fig, ax = plt.subplots(figsize=(11, 7))
ax.xaxis.set_major_formatter(formatter)
ax.plot(np.arange(pnl3.shape[0]), np.cumsum(pnl3))
fig.autofmt_xdate()
plt.show()

In [ ]:
annual_return(pnl3)

In [ ]:
annual_volatility(pnl3)

In [ ]:
annual_sharpe(pnl3)

In [ ]:
maximum_drawdown(pnl3)

In [ ]:
annual_turnover(actions3)

# 5. Out of Sample Test -- No transaction cost

In [ ]:
env_test = gym.make('Equity-outsample-0bps-v0')
env_test = DummyVecEnv([lambda: env_test])
model_test = PPO2.load("no_trans_cost_36dim_state_100000", env_test)

In [ ]:
pnl1, dates1, costs1, actions1 = evaluate(model_test, env_test, num_steps=40000)

In [ ]:
plt.style.use("ggplot")
formatter = MyFormatter(dates1)
fig, ax = plt.subplots(figsize=(11, 7))
ax.xaxis.set_major_formatter(formatter)
ax.plot(np.arange(pnl1.shape[0]), np.cumsum(pnl1))
fig.autofmt_xdate()
plt.show()

In [ ]:
annual_return(pnl1)

In [ ]:
annual_volatility(pnl1)

In [ ]:
annual_sharpe(pnl1)

In [ ]:
maximum_drawdown(pnl1)

In [ ]:
annual_turnover(actions1)

# 6. Out of Sample Test -- 2bps transaction cost

In [ ]:
env_test_2 = gym.make('Equity-outsample-2bps-v0')
env_test_2 = DummyVecEnv([lambda: env_test_2])
model_test_2 = PPO2.load("2bps_trans_cost_36dim_state_250000", env_test_2)

In [ ]:
pnl2, dates2, costs2, actions2 = evaluate(model_test_2, env_test, num_steps=40000)

In [ ]:
plt.style.use("ggplot")
formatter = MyFormatter(dates2)
fig, ax = plt.subplots(figsize=(11, 7))
ax.xaxis.set_major_formatter(formatter)
ax.plot(np.arange(pnl2.shape[0]), np.cumsum(pnl2))
fig.autofmt_xdate()
plt.show()

In [ ]:
annual_return(pnl2)

In [ ]:
annual_volatility(pnl2)

In [ ]:
annual_sharpe(pnl2)

In [ ]:
maximum_drawdown(pnl2)

In [ ]:
annual_turnover(actions2)